In [5]:
#Importar bibliotecas
import pandas as pd
import numpy as np
import psycopg2 as pg2

In [6]:
#select fullvisitorid, 
#SUM(revenue) AS Receita,
#COUNT(fullvisitorid) AS qt_de_transações,
#AVG(revenue) AS media_por_transação
#FROM hits
#GROUP BY (fullvisitorid)

In [7]:
#Encontrando receita, quantidade de transações e receita média por transação
#Conectando com o PostgreSQL
conn = pg2.connect(database='DP6', user='postgres', password='200613')
cur=conn.cursor()
#Query no PostgreSQL
cur.execute('''
        SELECT fullvisitorid, 
        SUM(revenue) AS Receita,
        COUNT(fullvisitorid) AS qt_de_transações,
        AVG(revenue) AS media_por_transação
        FROM hits
        GROUP BY (fullvisitorid)
        ''')
        #Fechando conexão
data_base=cur.fetchall()
parte1=pd.DataFrame(data_base)
cur.close()
conn.commit()

In [8]:
#Encontrando número de sessões com conversão
#Conectando com o PostgreSQL
conn = pg2.connect(database='DP6', user='postgres', password='200613')
cur=conn.cursor()
#Query no PostgreSQL
cur.execute('''
    SELECT hits.fullvisitorid, 
    COUNT(hits.fullvisitorid) AS qt_de_sessões_conversão
    FROM hits
    INNER JOIN sessions
    ON hits.fullvisitorid = sessions.fullvisitorid
    GROUP BY hits.fullvisitorid
    ''')
#Fechando conexão
data_base=cur.fetchall()
parte2=pd.DataFrame(data_base)
cur.close()
conn.commit()

In [ ]:
'''SELECT *
FROM (SELECT fullvisitorid, 
	SUM(revenue) AS Receita,
	COUNT(fullvisitorid) AS qt_de_transações,
	AVG(revenue) AS media_por_transação
	FROM hits
	GROUP BY (fullvisitorid)) 
	AS q0
INNER JOIN (SELECT hits.fullvisitorid, 
	COUNT(hits.fullvisitorid) AS qt_de_sessões_conversão
	FROM hits
	INNER JOIN sessions
	ON hits.fullvisitorid = sessions.fullvisitorid
	GROUP BY hits.fullvisitorid) AS q1
	ON q0.fullvisitorid = q1.fullvisitorid'''